In [1]:
# Run if working locally
%load_ext autoreload
%autoreload 2
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import sqlite3
from sqlite3 import Error
import pickle
import os, sys
import config

config.root_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.insert(0, config.root_path)

from db.dbv2 import Table, AugmentedTable, TrainTestTable

<IPython.core.display.Javascript object>

In [3]:
from src.dataset.gpt_augmentor import Augmentor

Metal device set to: Apple M1 Max


2022-12-30 17:20:13.813491: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-12-30 17:20:13.813738: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
All model checkpoint layers were used when initializing TFGPT2LMHeadModel.

All the layers of TFGPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


<IPython.core.display.Javascript object>

In [4]:
import numpy as np
import nltk
from nltk.tokenize import word_tokenize

<IPython.core.display.Javascript object>

In [5]:
dataset_type = "committee"

table = Table(dataset_type)
augmented_table = AugmentedTable(dataset_type)
train_test_table = TrainTestTable(dataset_type)

<IPython.core.display.Javascript object>

In [6]:
min_sent_tokens = 8
max_sent_tokens = 64

target_sentences_original = table.get_target_sentences()
target_sentences = [s[1] for s in target_sentences_original]
cleaned_target_sentences = []

for s in target_sentences:
    if len(word_tokenize(s)) > min_sent_tokens:
        shortened_sentence = " ".join(word_tokenize(s)[:max_sent_tokens])
        cleaned_target_sentences.append(shortened_sentence)

augmented_segments = Augmentor.augment_gpt2(
    cleaned_target_sentences[:2],
    fast=True,
    # multiply by 5 to account for 5 as a max segment
    max_seq_word_length=max_sent_tokens * 5,
    verbose=True,
)

Completed augmenting 1/2...
Completed augmenting 2/2...


<IPython.core.display.Javascript object>

In [ ]:
cleaned_target_sentences[:1], augmented_segments[:1]